<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(hash_enc_logging.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### Hashing, Encryption and Logging For Our Models   

### Lab Objectives:   
- Establish various mechanisms for securing our models once deployed  
- Leverage encryption for ensuring the privacy of our models  
- Leverage Hashing to ensure integrity of our models   
- Leverage continuous monitoring, to ensure we are able to detect what we cannot prevent

With that in mind, let us get some data and create a simple model.  

### Step 1:  


In [1]:
# Let us first start with import dotenv
from dotenv import dotenv_values
import hashlib

import cryptography
from cryptography.fernet import Fernet
import os
import logging

import onnxruntime
from onnxruntime import InferenceSession
import socket

import getpass
import sklearn
from sklearn.datasets import make_blobs
from sklearn.svm import LinearSVC
import joblib

In [2]:
### Version of key libraries used  
print(f'Cryptography version used:  {cryptography.__version__}')
print(f'Onnxruntime version used:  {onnxruntime.__version__}')
print(f'Sklearn version used:  {sklearn.__version__}')


Cryptography version used:  45.0.5
Onnxruntime version used:  1.22.1
Sklearn version used:  1.7.0


In [3]:
# Let us create a simple model for this problem.
X, y = make_blobs(centers=2, random_state=10)
print(f'Sample X: \n{X[:5]}')
print(f'Sample y: \n{y[:5]}')

Sample X: 
[[ 3.67982584  4.68165164]
 [ 2.94713748  4.46116773]
 [ 4.32968132  5.64396726]
 [ 6.73488595 -9.38994773]
 [ 2.44301805  3.84652646]]
Sample y: 
[1 1 1 0 1]


In [4]:
# Train the model
# Just a simple model. Nothing important with LinearSVC
clf = LinearSVC(random_state=10).fit(X, y)

# Save the model to disk
joblib.dump(value=clf, filename=r'/tmp/linear_svc_clf.joblib')

# Verify the model was saved
!ls /tmp/linear_svc_clf.joblib

/tmp/linear_svc_clf.joblib


With the model saved, let us setup our logger. In this case, we create a log file *securitynik_adversarial_ai.log* where we can write the interactions with our model. This file can also be forwarded to your SIEM or even analyzed locally, to understand the type of interactions being had with your models. 

### Step 2: 

In [5]:
# Let's setup a logger
# https://docs.python.org/3/howto/logging-cookbook.html
# There are a couple of different approaches we can use here

logger = logging.getLogger(name='SecurityNik_Adversarial_AI')
logger.setLevel(level=logging.DEBUG)

# Let's start this off with logging to a local file
# you can even continue to use this with your syslog or some other tool to forward this file to a remote logging destination
# Think about your SIEM for example.
log_file_handler = logging.FileHandler(filename=r'/tmp/securitynik_adversarial_ai.log')
log_file_handler.setLevel(level=logging.DEBUG)

# Setup the console logger, if we wanted to write to the screen
console_logger = logging.StreamHandler()
console_logger.setLevel(level=logging.DEBUG)
#logging.basicConfig(format='%(asctime)s %(clientip)-15s %(user)-8s %(message)s',  level=logging.INFO)

# Log remotely. Your SIEM?
remote_logger = logging.handlers.SysLogHandler(address=('127.0.0.1', 514), socktype=socket.SOCK_DGRAM)
remote_logger.setLevel(level=logging.DEBUG)

logger.addHandler(hdlr=log_file_handler)
logger.addHandler(hdlr=console_logger)
logger.addHandler(hdlr=remote_logger)

# Setup a logging format  
formatter = logging.Formatter(fmt='%(asctime)-15s %(name)-5s %(levelname)-8s %(message)s')
log_file_handler.setFormatter(fmt=formatter)
console_logger.setFormatter(fmt=formatter)

# With this information if you know in you know that only specific sources within your org should be making predictions, 
# Then you can look for deviations from this source IP, user, UID and platform
logger.info(msg=f'[*] Application startup: Hostname: {socket.gethostname()} / {socket.gethostbyaddr(socket.gethostname())[2][0]}, Running as user: {getpass.getuser()} with uid: {os.getuid()} {getpass.sys.platform} Platform')

2025-08-01 22:44:23,647 SecurityNik_Adversarial_AI INFO     [*] Application startup: Hostname: SECURITYNIK-G14 / 127.0.1.1, Running as user: securitynik with uid: 1000 linux Platform


In [6]:
# Take a peak into the log.
# You should do this from your command line
# We are doing it here for convenience.  
!cat /tmp/securitynik_adversarial_ai.log

2025-08-01 22:44:23,647 SecurityNik_Adversarial_AI INFO     [*] Application startup: Hostname: SECURITYNIK-G14 / 127.0.1.1, Running as user: securitynik with uid: 1000 linux Platform


To help with the encrypting and decrypting, we will use a library named cryptography. 

https://cryptography.io/en/latest/  

### Step 3:  

In [7]:
# Implement a function to encrypt the file
def encrypt_file(model_file=None):
    # Generate a key
    enc_key = Fernet.generate_key()
    fernet_enc = Fernet(key=enc_key)

    # Open the file we would like to read
    with open(file=model_file, mode='rb') as e_file:
        clear_text = e_file.read()

    # Perform the actual encryption
    cipher_text = fernet_enc.encrypt(clear_text)

    # Overwrite the file with the new encrypted data
    with open(file=model_file, mode='wb') as e_file:
        e_file.write(cipher_text)
    
    # Return the encryption key
    return enc_key

In [8]:
# Test the encryption function
# We are obviously assuming you have a correct file that you are pointing to
# We could have done error checking above
# However, we are keeping it simple
encrypt_file(model_file=r'/tmp/linear_svc_clf.joblib')

b'ZqVhZqfSi28EqysvPSAnp8sp47YCMZjE2MU97W33Ksg='

Let us work on the integrity checking at this time via hashing. 

### Step 4: 

In [9]:
# Create a function to hash a file
def hash_file(model_file=None):
    #print(f'Hashing file: {model_file}')

    # Open the file to be hashed
    with open(file=model_file, mode='rb') as hash_file:
        sha_256_hash = hashlib.sha256(string=hash_file.read())

    # Return the file hash
    return sha_256_hash.hexdigest()

In [10]:
# Test the hash function
hash_file(model_file=r'/tmp/linear_svc_clf.joblib')

'fa88efbda2cabe7c97cda75a6f3933e349c314823966e1107297be0d20f9c54d'

We put everything together now in one function.

### Step 5:  

In [11]:
# Finalize the file. 
# Actually call the functions to encrypt and hash
# We will also log this activity
def hash_encrypt(model_file=None, env_path='/tmp/.env'):
    # Define the path to the .env file
    env_file_path = os.path.abspath(path=env_path)

    # Check to see if the .env file exist in the current directory
    if os.path.exists(path=env_file_path):
        logger.info(msg='[*] Dot env file found')
    else:
        logger.warn(msg='[+] .env file not found. Creating it!')
        open(file=env_file_path, mode='wt')

    # Call our encrypt function
    e_key = encrypt_file(model_file=model_file)

    # Call our hash function
    h_file = hash_file(model_file=model_file)

    # Write the hash information to the .env file
    with open(file=env_file_path, mode='wt') as e_file:
        e_file.write(f'model_file_path={os.path.abspath(path=model_file)}\n')
        e_file.write(f'model_encryption_key={e_key}\n')
        e_file.write(f'model_sha_256_hash={h_file}\n')
        
    return

In [12]:
# Call the function to test
hash_encrypt(model_file=r'/tmp/linear_svc_clf.joblib')

/tmp/ipykernel_30831/1745664762.py:12: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(msg='[+] .env file not found. Creating it!')
2025-08-01 22:44:24,075 SecurityNik_Adversarial_AI WARNING  [+] .env file not found. Creating it!


In [13]:
# If above went well, we should now be able to see into the .env file
!cat /tmp/.env

model_file_path=/tmp/linear_svc_clf.joblib
model_encryption_key=b'80w-niZUn6QZnPmmyA1nNHAw049xICN-Mknc-Ff59r4='
model_sha_256_hash=906cab4bd6ec19335a98f982d51e33f1892cb8ad09bc6cbf7cea749dddd8990e


At this point, when the model is loaded, can validate its integrity and privacy by calling our functions. We can also log the interaction as expected. 

### Step 6:

In [14]:
# Now that you have the .env file populated
# all you need to do is load the model and validate everything
def load_model_make_prediction(model_file='', env_path=r'/tmp/.env'):
    # Define the path to the .env file
    env_values = dotenv_values(dotenv_path=os.path.abspath(path=env_path))
    
    # Validate the file is loading from the correct path
    if model_file == env_values['model_file_path']:
        logger.info(msg='[*] File path successfully validated ...')
    else:
        logger.warning(msg=f'[!] Model file **{model_file}** is being load from an unknown location')
    
    # Validate the hash of the encrypted file
    if hash_file(model_file=model_file) == env_values['model_sha_256_hash']:
        logger.info(msg='[*] Model File integrity successfully validate')
    else:
        logger.error(msg='[!] Exiting! File integrity check failed')
        return
    
    # If the file hash was successful validated, then decrypt the file
    # Notice the slicing at the end? This is because the result is stored as a bytes
    # We need to remove the b' and final '
    fernet_decrypt = Fernet(env_values['model_encryption_key'][2:-1])

    try:
        with open(file=model_file, mode='rb') as fp:
            # Decrypt the model
            loaded_model = fernet_decrypt.decrypt(fp.read())
            logger.info(msg=f'[*] Successfully decrypted model file: {model_file}')
                
    except Exception as e:
        # If an error is found then during decryption, catch the error
        logger.error(msg=f'[!] An error occurred while decrypting the file. Exiting ...')
        return

    # if everything worked out, then make a prediction
    logger.info(f'[*] Final Prediction: {clf.predict(X=X[:5])}')

    

In [15]:
# Call the function to make the predictions
load_model_make_prediction(model_file='/tmp/linear_svc_clf.joblib')

2025-08-01 22:44:24,408 SecurityNik_Adversarial_AI INFO     [*] File path successfully validated ...
2025-08-01 22:44:24,414 SecurityNik_Adversarial_AI INFO     [*] Model File integrity successfully validate
2025-08-01 22:44:24,418 SecurityNik_Adversarial_AI INFO     [*] Successfully decrypted model file: /tmp/linear_svc_clf.joblib
2025-08-01 22:44:24,424 SecurityNik_Adversarial_AI INFO     [*] Final Prediction: [1 1 1 0 1]


In [16]:
# Verify the file 
!cat /tmp/securitynik_adversarial_ai.log

2025-08-01 22:44:23,647 SecurityNik_Adversarial_AI INFO     [*] Application startup: Hostname: SECURITYNIK-G14 / 127.0.1.1, Running as user: securitynik with uid: 1000 linux Platform
2025-08-01 22:44:24,075 SecurityNik_Adversarial_AI WARNING  [+] .env file not found. Creating it!
2025-08-01 22:44:24,408 SecurityNik_Adversarial_AI INFO     [*] File path successfully validated ...
2025-08-01 22:44:24,414 SecurityNik_Adversarial_AI INFO     [*] Model File integrity successfully validate
2025-08-01 22:44:24,418 SecurityNik_Adversarial_AI INFO     [*] Successfully decrypted model file: /tmp/linear_svc_clf.joblib
2025-08-01 22:44:24,424 SecurityNik_Adversarial_AI INFO     [*] Final Prediction: [1 1 1 0 1]


### Takeaways:   
- We were able to implement integrity, encryption and logging for our model   
- These are critical components we need to consider when deploying our models  
- We are here for adversarial AI purposes and thus we should understand the importance of these tasks  